ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
https://donlapark.pages.dev/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [ ]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

--2023-12-21 04:48:31--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.44.200, 172.66.47.56, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.44.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv.1’

Auto.csv.1          100%[===================>]  17.66K  --.-KB/s    in 0s      

2023-12-21 04:48:31 (133 MB/s) - ‘Auto.csv.1’ saved [18079/18079]



In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [ ]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย `dataframe.to_numpy()`

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ `mpg`
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ `auto_df.drop`)


In [ ]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

In [ ]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 19. ,
       16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. , 13. ,
       18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. , 27. ,
       26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. , 17. ,
       11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. , 22. ,
       21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. , 14. ,
       15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. , 18. ,
       18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. , 22. ,
       18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. , 15. ,
       24. , 20. , 11. , 20. , 19. , 15. , 31. , 26. , 32. , 25. , 16. ,
       16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. , 26. , 31. ,
       32. , 28. , 24. , 26. , 24. , 26. , 31. , 19

In [ ]:
X

array([[   8. ,  307. ,  130. , 3504. ,   12. ,   70. ],
       [   8. ,  350. ,  165. , 3693. ,   11.5,   70. ],
       [   8. ,  318. ,  150. , 3436. ,   11. ,   70. ],
       ...,
       [   4. ,  135. ,   84. , 2295. ,   11.6,   82. ],
       [   4. ,  120. ,   79. , 2625. ,   18.6,   82. ],
       [   4. ,  119. ,   82. , 2720. ,   19.4,   82. ]])

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [ ]:
X.shape

(392, 6)

In [ ]:
#print([1]*392)
#print(np.ones(392))

In [ ]:
ones =  [1]*X.shape[0] #your answer here

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [ ]:
#เช็คคำตอบ
print(X)

[[1.000e+00 8.000e+00 3.070e+02 ... 3.504e+03 1.200e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.500e+02 ... 3.693e+03 1.150e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.180e+02 ... 3.436e+03 1.100e+01 7.000e+01]
 ...
 [1.000e+00 4.000e+00 1.350e+02 ... 2.295e+03 1.160e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.200e+02 ... 2.625e+03 1.860e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.190e+02 ... 2.720e+03 1.940e+01 8.200e+01]]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred`

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [ ]:
# Let's start by implementing linear regression from scratch
# using the formula provided in the class

def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    # TODO: your code here
    inverse = np.linalg.inv((X.T)@X)
    cal = (X.T)@(y)
    beta = inverse @ cal
    return beta


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น"""
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here
    y_pred = X@beta

    return y_pred

In [ ]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

print("Coefficients are:", beta)
print("Predictions are:", y_pred)

Coefficients are: [-1.45352505e+01 -3.29859089e-01  7.67843024e-03 -3.91355574e-04
 -6.79461791e-03  8.52732469e-02  7.53367180e-01]
Predictions are: 0      15.082919
1      14.072575
2      15.536315
3      15.534475
4      15.286407
         ...    
392    29.336024
393    34.271065
394    32.320658
395    30.562127
396    29.976004
Length: 392, dtype: float64


#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [ ]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations:', n)
print('Number of variables:', p)

Number of observations: 392
Number of variables: 6


## 2. F-statistic

### Exercise 2: เขียนสมมติฐานหลัก ($H_0$) และสมมติฐานแย้ง ($H_a$) ที่ใช้ F-statistic ในการทดสอบ

### Exercise 3: จากเวกเตอร์ของ outcomes `y` คำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$

$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [ ]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS = np.sum(np.square(y-y_mean)) # TODO: your code here
print(TSS)
RSS = np.sum(np.square(y-y_pred))
print(RSS)

23818.99346938776
4543.347024714769


คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [ ]:
F = ((TSS-RSS)/p)/(RSS/(n-p-1))

print(F)

272.23409822574575


In [ ]:
# Compute p_value of the F-statistic
# p-value > 0.05 -> Accept H0
# p-value <= 0.05 -> Accept Ha
# p-value ต้องอยู่ระหว่าง 0 กับ 1 เท่านั้น

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

3.792336100722869e-135


### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรจากค่า p-value ที่ได้ข้างบน

### คำตอบของ Exercise 5:
ค่า p-value มีค่าน้อยมากซึ่งน้อยกว่าหรือเท่ากับ 0.05 จึงตกอยู่ทางฝั่งขวาของกราฟ เราจึงยอมรับสมมติฐาน Ha หมายความว่ามีตัวแปรต้นอย่างน้อยหนึ่งตัวที่มีความสัมพันธ์กับตัวแปรตาม



## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*}
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

In [ ]:
def t_to_p_values(t_statistics):
    """t_statistics: numpy array ของค่า t-statistic ของตัวแปรต่างๆ
    Return: เวกเตอร์ของค่า p-value ของค่า t-statistic ที่ให้มา"""
    return stats.t.sf(np.abs(t_statistics), n-p-1)*2



### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อดึงค่าในแนวทแยงของ `A`

In [ ]:
# TODO: your code here

RSE = np.sqrt(RSS/(n-p-1))  # vector
inverse = np.linalg.inv((X.T)@X)
SE = RSE*(np.sqrt(inverse.diagonal()))  # vector
t = beta/SE  # vector ของสัมประสิทธิ์ทั้งหมด

# p-value of each variable in X
p = t_to_p_values(t)
print(t)
print(p)

[ -3.07481008  -1.00054285   1.05129121  -0.02846815 -10.21959159
   0.84177638  14.42876473]
[2.25426691e-03 3.17667163e-01 2.93774645e-01 9.77303335e-01
 6.90853599e-22 4.00428213e-01 3.88925817e-38]


list ชื่อตัวแปร

In [ ]:
print("List of predictors:", predictor_names[1:-2])

List of predictors: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'year'],
      dtype='object')


### คำตอบของ Exercise 6:





## 4. Variable selection

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [ ]:
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1)
X = sm.add_constant(X)     # add bias constant
y = auto_df['mpg']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 21 Dec 2023   Prob (F-statistic):          3.79e-135
Time:                        04:53:54   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   

In [67]:
import statsmodels.formula.api as smf

model2 = smf.ols('''mpg ~ cylinders + displacement + horsepower
                        + weight + acceleration + year''', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 21 Dec 2023   Prob (F-statistic):          3.79e-135
Time:                        05:43:09   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

### Exercise 7: ใช้ Backward selection ในการเลือกตัวแปร โดยที่เราจะหยุดเมื่อ p-value สของตัวแปรทุกตัว < 0.05 แล้วเขียนสมการของโมเดลที่ได้

In [ ]:
# ลบ ตัวแปรต้นที่มีค่า p-value สูงสุดจนกว่าจะเหลือแต่ p-value ที่ < 0.05

In [ ]:
# Block for running your own code
model2 = smf.ols('mpg ~ weight + year', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     819.5
Date:                Thu, 21 Dec 2023   Prob (F-statistic):          3.33e-140
Time:                        05:12:30   Log-Likelihood:                -1037.6
No. Observations:                 392   AIC:                             2081.
Df Residuals:                     389   BIC:                             2093.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -14.3473      4.007     -3.581      0.0

In [68]:
# If you want to write with for-loop
#print(model2.tvalues)
print(model2.pvalues)
#print("t-value of weight:", model2.tvalues["weight"])
print("p-value of weight:", model2.pvalues["weight"])

Intercept       2.437741e-03
cylinders       3.212169e-01
displacement    2.973318e-01
horsepower      9.774501e-01
weight          1.416190e-21
acceleration    4.038303e-01
year            1.410428e-37
dtype: float64
p-value of weight: 1.416189889352751e-21


### คำตอบของ Exercise 7:

$$
y = (-14.3473)+(-0.0066)*(weight)+(0.7573)*(year)
$$